In [1]:
import cv2
print('opencv: ' +  cv2.__version__)

import numpy as np
import math

from Odometry2D import VisualOdometry
from OdometryUtil import PinholeCamera, truncate

opencv: 3.4.3


### Camera settings

In [2]:
cam = PinholeCamera(1241.0, 376.0, 718.8560, 718.8560, 607.1928, 185.2157)

### Output settings

In [3]:
OUT_W = 1000
OUT_H = 1000

START_X = 500
START_Y = 500

RES_W = 1241
RES_H = 376

traj = np.zeros((OUT_W, OUT_H, 3), dtype=np.uint8)

font = cv2.FONT_HERSHEY_TRIPLEX
fontColor = (255, 255, 0)

### Dataset settings

In [4]:
DATASET = '07'

PATH_POS = 'D:/Data/data_odometry_pos/poses/'
PATH_COLOR = 'D:/Data/data_odometry_color/dataset/sequences/'
PATH_GRAY = 'D:/Data/data_odometry_gray/dataset/sequences/'

### Visual Odometry settings

SCALE_FIXED = 0
SCALE_DISTANCE = 1
SCALE_TIME = 2

In [5]:
mode = 2

vo = VisualOdometry(cam, mode)
vo.readPosition(PATH_POS + DATASET + '.txt')
vo.readTime(PATH_GRAY + DATASET + '/times.txt')

Operation mode: SCALE_DISTANCE


### Visual Odometry

In [6]:
for img_id in range(vo.posLength):
    time1 = cv2.getTickCount()
    imgC = cv2.imread(PATH_COLOR + DATASET + '/image_2/' + str(img_id).zfill(6) + '.png', cv2.IMREAD_COLOR)
    imgG = cv2.imread(PATH_GRAY + DATASET + '/image_0/' + str(img_id).zfill(6) + '.png', cv2.IMREAD_GRAYSCALE)

    vo.update(imgG, img_id)
    
    cur_t = vo.cur_t
    if(img_id > 2):
        x, y, z = cur_t[0], cur_t[1], cur_t[2]
    else:
        x, y, z = 0., 0., 0.
    draw_x, draw_y = int(x)+START_X, int(z)+START_Y
    true_x, true_y = int(vo.trueX)+START_X, int(vo.trueZ)+START_Y

    cv2.circle(traj, (true_x,true_y), 2, (0, img_id*255/vo.posLength, 255-img_id*255/vo.posLength), 1)
    cv2.circle(traj, (draw_x,draw_y), 1, (img_id*255/vo.posLength, 255-img_id*255/vo.posLength, 0), 1)
    
    time2 = cv2.getTickCount()
    time = math.floor(((time2 - time1) * 1000) / cv2.getTickFrequency())
    imgOut = cv2.copyMakeBorder(imgC, 0, 40, 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    text1 = " Coord: x=" + str(truncate(x,3)) + "m y=" + str(-1 * truncate(z,3)) + "m"
    text2 = " Scale:" + str(vo.scale)
    textString = str(time) + " ms" + text1 + text2
    cv2.putText(imgOut, textString, (10, RES_H + 30), font, 1, fontColor, 2, 8)

    cv2.imshow('Road facing camera', imgOut)
    cv2.imshow('Trajectory', traj)
    cv2.waitKey(1)

cv2.imwrite('map.png', traj)

while(True):
    getKey = cv2.waitKey(10) & 0xFF
    if getKey == ord('q'):
        cv2.destroyAllWindows()
        break

KeyboardInterrupt: 